# Interactive Editor for 'metarisk_releases_dim' (Enhanced)

This notebook provides a user-friendly interface to directly edit records in the `gc_prod_sandbox.su_eric_regna.metarisk_releases_dim` table. **This version includes advanced editors (multi-select, date picker) and is optimized for Databricks Runtime 16.4 LTS.**

### Usage Guide

*   **Author:** regnaer
*   **Generated:** 2025-10-01 02:24:08 UTC
*   **Purpose:** This notebook provides a user-friendly interface to directly edit records in the `gc_prod_sandbox.su_eric_regna.metarisk_releases_dim` table.

**How to Use This Editor:**

1.  **Run All Cells:** Click "Run All" to install libraries, restart the Python kernel (this is required), configure the connection, and display the interactive grid below, which will be populated with live data.
2.  **Edit Data in the Grid:**
    *   **Dropdowns:** Double-click a cell in the `Status` or `Product` columns to see a dropdown.
    *   **Date Picker:** Double-click a cell in the `start_date` column to use a calendar.
    *   **Multi-Select (`release_labels`):** Double-click a cell in the `release_labels` column. A popup with checkboxes will appear. Select, deselect, or clear items and click outside the popup to confirm.
    *   **Multi-Line Text (`Callouts`):** Double-click a cell in the `Callouts` column. A larger text box will appear. You can use `Enter` to create new lines.
3.  **Save Your Changes:**
    *   After making all your edits, you **MUST** click the green **"Save Changes"** button.
4.  **Refresh Data:**
    *   Click the blue **"Refresh Data"** button to discard any unsaved changes in the grid and reload the latest data from the Databricks table.

## 1. Installation and Setup

In [ ]:
# Installs the latest version of ipyaggrid, compatible with DBR 16.4 LTS.
%pip install ipyaggrid==0.5.4

In [ ]:
# This command restarts the Python kernel to ensure the newly installed library is used.
# It is a required step after a %pip install command in Databricks.
dbutils.library.restartPython()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

from ipyaggrid import Grid
from ipywidgets import Button, HBox, VBox, HTML, Output
from IPython.display import display

# The SparkSession is automatically available in a Databricks notebook environment
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## 2. Configuration for `metarisk_releases_dim`

In [ ]:
# --- REAL CONFIGURATION --- #
CATALOG = "gc_prod_sandbox"
SCHEMA = "su_eric_regna"
TABLE = "metarisk_releases_dim"
TABLE_FQN = f"{CATALOG}.{SCHEMA}.{TABLE}"

# Define all columns you want to display in the grid
COLUMNS_TO_DISPLAY = ["surrogate_key", "title", "Status", "Product", "start_date", "release_labels", "Callouts"]
PRIMARY_KEY = "surrogate_key"

## 3. Data Loading and Sample Data Generation

This section contains two functions: one to generate sample data for quick testing, and one to load the actual data from your Databricks table. By default, the live data is used.

In [ ]:
def load_data_from_databricks(table_fqn, columns):
    """Loads data from the specified Databricks table into a pandas DataFrame."""
    print(f"Loading data from {table_fqn}...")
    try:
        # Select specific columns and limit to 200 rows as per the requirement
        df_spark = spark.table(table_fqn).select(*columns).limit(200)
        df_pandas = df_spark.toPandas()
        
        # Ensure date columns are in string format for the grid editor
        if 'start_date' in df_pandas.columns:
            # Coerce to datetime and format, handling potential errors
            df_pandas['start_date'] = pd.to_datetime(df_pandas['start_date'], errors='coerce').dt.strftime('%Y-%m-%d')
            # Fill any NaT values that may have resulted from coerce
            df_pandas['start_date'] = df_pandas['start_date'].fillna('')
        
        print("Data loaded successfully.")
        return df_pandas
    except Exception as e:
        print(f"ERROR: Could not load data from {table_fqn}. Please check table name, column names, and permissions. Error: {e}")
        return pd.DataFrame(columns=columns)

# --- CHOOSE DATA SOURCE --- #
df = load_data_from_databricks(TABLE_FQN, COLUMNS_TO_DISPLAY)

## 4. Configure Grid Columns with Custom Editors

In [ ]:
# This JavaScript defines the custom multi-select checkbox editor.
# It is passed to the Grid object and evaluated in the browser.
custom_js = """
class MultiSelectCellEditor {
    init(params) {
        this.params = params;
        // ADAPTED: Split the semicolon-separated string into an array
        this.value = params.value ? String(params.value).split(';') : [];

        this.eGui = document.createElement('div');
        this.eGui.style.backgroundColor = 'white';
        this.eGui.style.border = '1px solid #ccc';
        this.eGui.style.padding = '8px';
        this.eGui.style.maxHeight = '200px';
        this.eGui.style.overflowY = 'auto';

        const values = params.values || []; 
        values.forEach(option => {
            const container = document.createElement('div');
            const checkbox = document.createElement('input');
            checkbox.type = 'checkbox';
            checkbox.id = params.colDef.field + '_' + option;
            checkbox.value = option;
            if (this.value.includes(option)) {
                checkbox.checked = true;
            }

            const label = document.createElement('label');
            label.htmlFor = checkbox.id;
            label.innerText = ' ' + option;
            
            checkbox.addEventListener('change', () => {
                if (checkbox.checked) {
                    this.value.push(option);
                } else {
                    this.value = this.value.filter(v => v !== option);
                }
            });
            
            container.appendChild(checkbox);
            container.appendChild(label);
            this.eGui.appendChild(container);
        });
    }

    getGui() { return this.eGui; }
    
    getValue() { 
        // ADAPTED: Join the array back into a semicolon-separated string for the backend
        return this.value.join(';');
    }

    isPopup() { return true; }
}
"""

In [ ]:
def configure_grid_columns():
    """Configure grid columns with appropriate editors and settings for the metarisk table."""
    return [
        {'headerName': 'SK', 'field': 'surrogate_key', 'editable': False, 'width': 120},
        {'headerName': 'Title', 'field': 'title', 'editable': False, 'width': 180},
        {
            'headerName': 'Status',
            'field': 'Status',
            'editable': True,
            'cellEditor': 'agSelectCellEditor',
            'cellEditorParams': {'values': ["On-Track", "At-Risk", "Off-Track", "Complete", "Not Started", "Blocked"]},
            'width': 130
        },
        {
            'headerName': 'Product',
            'field': 'Product',
            'editable': True,
            'cellEditor': 'agSelectCellEditor',
            'cellEditorParams': {'values': ["MR Desktop", "Data", "MR Online", "MR Live", "Support", "MR Rate", "AI"]},
            'width': 130
        },
        {
            'headerName': 'Start Date',
            'field': 'start_date',
            'editable': True,
            'cellEditor': 'agDateCellEditor', # Re-enabled the date picker
            'width': 130
        },
        {
            'headerName': 'Release Labels',
            'field': 'release_labels',
            'editable': True,
            'cellEditor': 'MultiSelectCellEditor', # Reference the custom editor
            'cellEditorParams': {
                'values': ["Engine Work", "Global Tools", "Hotfix", "Modelbuilder", "Rating Tools", "Reinsurance", "Templates", "UI/UX", "Waterfalls"]
            },
            # This makes the display prettier (e.g., "Engine, UI/UX")
            'valueFormatter': "params.value ? params.value.replace(/;/g, ', ') : ''",
            'width': 250
        },
        {
            'headerName': 'Callouts',
            'field': 'Callouts',
            'editable': True,
            'cellEditor': 'agLargeTextCellEditor',
            'cellEditorParams': {'rows': 5, 'cols': 40},
            'cellStyle': {'white-space': 'pre-wrap'},
            'autoHeight': True,
            'width': 350
        }
    ]


## 5. Create and Display the Grid

In [ ]:
column_defs = configure_grid_columns()

grid_options = {
    'columnDefs': column_defs,
    'defaultColDef': {
        'filter': True,
        'sortable': True,
        'resizable': True
    },
    'enableCellChangeFlash': True,
    'undoRedoCellEditing': True,
    'stopEditingWhenCellsLoseFocus': True
}

grid = Grid(
    grid_data=df,
    grid_options=grid_options,
    js_helpers_custom=custom_js, # Pass the custom JS to the grid
    height=500,
    width='100%',
    theme='ag-theme-balham',
    quick_filter=True,
    show_toggle_edit=True,
    sync_on_edit=True, 
    export_csv=True,
    export_excel=True
)

## 6. Handle Data Changes and Save to Databricks

This section implements the robust "manual save" pattern. When the user clicks "Save Changes", we pull the complete dataset from the grid and then perform the `MERGE` operation against your live table.

In [ ]:
output_area = Output()
save_button = Button(description="Save Changes", button_style="success", icon="save")
refresh_button = Button(description="Refresh Data", button_style="info", icon="refresh")
status_html = HTML("<p>Grid is ready. Edit cells and click 'Save Changes'.</p>")

def on_save_clicked(b):
    with output_area:
        output_area.clear_output()
        status_html.value = f"<p style='color:blue'>Saving... Please wait.</p>"
        try:
            # Get the updated data from the 'grid' key of the grid_data_out dictionary.
            updated_df = grid.grid_data_out['grid']
            
            print("Data to be saved:")
            display(updated_df.head())
            
            # --- DATABRICKS INTEGRATION --- #
            print(f"\nConverting to Spark DataFrame and merging into {TABLE_FQN}...")
            updated_spark_df = spark.createDataFrame(updated_df)
            updated_spark_df.createOrReplaceTempView("temp_updates_for_merge")
            
            # Construct the MERGE statement dynamically based on editable columns
            editable_cols = [c['field'] for c in configure_grid_columns() if c.get('editable')]
            set_clauses = ",\n".join([f"  target.{col} = source.{col}" for col in editable_cols])
            
            merge_sql = f"""
            MERGE INTO {TABLE_FQN} AS target
            USING temp_updates_for_merge AS source
            ON target.{PRIMARY_KEY} = source.{PRIMARY_KEY}
            WHEN MATCHED THEN UPDATE SET
            {set_clauses}
            """
            
            print("Executing MERGE statement:")
            print(merge_sql)
            spark.sql(merge_sql)
            # ------------------------------ #
            
            status_html.value = f"<p style='color:green'>Data saved to {TABLE_FQN} successfully at {datetime.now().strftime('%H:%M:%S')}</p>"
            print("\nSave operation completed.")

        except Exception as e:
            error_message = f"Error saving data: {str(e)}"
            print(error_message)
            status_html.value = f"<p style='color:red'>{error_message}</p>"

def on_refresh_clicked(b):
    with output_area:
        output_area.clear_output()
        status_html.value = f"<p style='color:blue'>Refreshing data from {TABLE_FQN}...</p>"
        try:
            refreshed_df = load_data_from_databricks(TABLE_FQN, COLUMNS_TO_DISPLAY)
            # Use update_grid_data with a positional argument.
            grid.update_grid_data(refreshed_df)
            status_html.value = f"<p style='color:green'>Grid data refreshed at {datetime.now().strftime('%H:%M:%S')}</p>"
        except Exception as e:
            error_message = f"Error refreshing data: {str(e)}"
            print(error_message)
            status_html.value = f"<p style='color:red'>{error_message}</p>"

save_button.on_click(on_save_clicked)
refresh_button.on_click(on_refresh_clicked)

controls = VBox([
    HBox([save_button, refresh_button]),
    status_html,
    output_area
])

display(VBox([grid, controls]))